# Robot With Motion Planner

The robot with motion planner slightly extends the robot interface presented in the first tutorial, by adding motion planning capabilities based on the motion planner introduced in tutorial 2. This is basically a combination of them with few more features. To create an instance, a motion planner should be provided. Note that it can be an extension of the motion planner that you can implement with different geometry as shown in tutorial 2. The controller is agnostic to your geometry and motion planner implementation, as long as it implements the abstract motion planner interface.

In [1]:
from ur_lab.manipulation.robot_with_motion_planning import RobotInterfaceWithMP
from ur_lab.robot_inteface.robots_metadata import ur5e_1

# from previous tutorial:
from ur_lab.motion_planning.motion_planner import MotionPlanner
from ur_lab.motion_planning.geometry_and_transforms import GeometryAndTransforms

mp = MotionPlanner()
gt = GeometryAndTransforms.from_motion_planner(mp)
robot = RobotInterfaceWithMP(ur5e_1["ip"], ur5e_1["name"], mp, gt)

# shortcut for convinience:
# robot = RobotInterfaceWithMP.build_from_robot_name_and_ip(ur5e_1["ip"], ur5e_1["name"])

RobParser: Reading robot file /home/clair01/PycharmProjects/clair-robotics-stack/ur_lab/motion_planning/ur5e_rob/ur5e_hires.rob...
RobParser:    Parsing robot file, 8 links read...
RobParser: Loaded geometries in time 0.009056s, 11028 total primitive elements
RobParser: Done loading robot file /home/clair01/PycharmProjects/clair-robotics-stack/ur_lab/motion_planning/ur5e_rob/ur5e_hires.rob
RobParser: Reading robot file /home/clair01/PycharmProjects/clair-robotics-stack/ur_lab/motion_planning/ur5e_rob/ur5e_hires.rob...
RobParser:    Parsing robot file, 8 links read...
RobParser: Loaded geometries in time 0.025287s, 11028 total primitive elements
RobParser: Done loading robot file /home/clair01/PycharmProjects/clair-robotics-stack/ur_lab/motion_planning/ur5e_rob/ur5e_hires.rob
*** klampt.vis: using GLUT as the visualization backend ***
***      Some functionality may not be available!       ***
GLUTWindowManager.createWindow: window title robots_visualization , id 1
GLUTWindowManager.sho

/home/clair01/PycharmProjects/clair-robotics-stack/.venv/lib/python3.10/site-packages/klampt/vis/visualization.py:704: UserWarning: klampt.visualization: QT is not available, falling back to poorer
GLUT interface.  Returning to another GLUT thread will not work
properly.
  warnings.warn("klampt.visualization: QT is not available, falling back to poorer\nGLUT interface.  Returning to another GLUT thread will not work\nproperly.")


plan_and_moveJ method moves to a specified config, and if there is no direct path available already handles the motion planning for you. Note that there is a plan_and_move_home method that you can use for better safety than move_home.

In [34]:
q1 = [0.03923, -1.76654, -2.27161, -2.19254, 0.40798, 3.07572]
q2 = [-0.06798, -1.84265, -2.65451, 1.33179, 1.61574, 1.64959]

robot.plan_and_move_home()
robot.plan_and_moveJ(q1)

# No direct path available, motion planning is needed:
robot.plan_and_moveJ(q2)

planning motion...
planning took  0.8056280612945557  seconds.


True

This interface also has another method that may be helpful, plan_and_move_to_xyzrz will move the end effector to the desired cartesian coordinates with the gripper facing down, rotated around z axis by rz. This is useful for some pick and place tasks. It can be be followed by a linear movement downward, or even move_until_contact to pick an object.

In [42]:
robot.plan_and_move_home()

import numpy as np
robot.plan_and_move_to_xyzrz(x=0.4, y=0.0, z=0.2, rz=0)
robot.plan_and_move_to_xyzrz(x=0.5, y=-0.2, z=0.2, rz=-np.pi/2)
robot.plan_and_move_to_xyzrz(x=0.5, y=-0.2, z=0.2, rz=-3*np.pi/4)

True

== Note: x, y, z are in world coordinates and not in robot base coordinates! ==

# Manipulation Interface

In [3]:
del robot

In [5]:
robot.move_home()